In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels

In [2]:
import urllib
params ='DRIVER={SQL Server};SERVER=CL-AMILAH\AMILA;DATABASE=GSH;UID=Amila;Trusted_Connection=yes;'       
params = urllib.parse.quote_plus(params)

from sqlalchemy import create_engine
db = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

In [3]:
sql = '''
SELECT inv.[ID] ,inv.SiteUID, [Date] , [OutletUID] ,ot.Name AS outlet ,ot.ProvinceUID,pr.Description AS province ,ot.DistrictUID ,  di.Description AS district 
, ot.AreaUID , ar.Description AS Area , inv.[AgentUID] ,Concat(emp.FirstName ,' ' , emp.MiddleInitials , ' ', emp.LastName ) AS AgentName , inv.[DistributorUID] , d.Name AS distributor, [ItemUID] ,it.Description AS item, [UnitQty] 
, [UnitPrice] , [RetailUnitPrice] , [FreeQty] , [FreeCompanyQty] , [GrossValue]
,datename(YEAR, Date) AS Year
,datename(month, Date) AS MonthName
,MONTH(Date) AS Month
,datename(DAY, Date) AS Day
,DATEPART(DW,Date) AS WeekDay
,datename(WEEKDAY, Date) AS DateName
,DATEPART (hh,Date) as Hour

  FROM [GSH].[dbo].[vw_Invoice] inv
  LEFT JOIN [GSH].[dbo].[vw_InvoiceLine] invl
  ON inv.siteUID = invl.TxnInvoice_SiteUID AND inv.uid = invl.TxnInvoiceUID	
  LEFT JOIN [GSH].[dbo].[Item] it
  ON invl.ItemUID = it.UID
  LEFT JOIN [GSH].[dbo].[Outlet] ot
  ON inv.OutletUID = ot.UID
  LEFT JOIN [GSH].[dbo].[Area] pr
  ON ot.ProvinceUID = pr.UID
  LEFT JOIN [GSH].[dbo].[Area] di
  ON ot.DistrictUID = di.UID
  LEFT JOIN [GSH].[dbo].[Area] ar
  ON ot.AreaUID = ar.UID
  LEFT JOIN [GSH].[dbo].[Distributor] d
  ON inv.DistributorUID = d.UID
  LEFT JOIN [GSH].[dbo].[avLogin] avlg
  ON inv.AgentUID = avlg.BusinessUserUID
  LEFT JOIN [GSH].[dbo].[Employee] emp
  ON avlg.EmployeeUID = emp.UID
  WHERE inv.mpt_TypeEnum IN ( 21 , 44, 14, 1) AND inv.ExpiryDate IS NULL AND inv.SubmittedDate IS NOT NULL;
'''

invoice_All = pd.read_sql_query(sql, db)

In [4]:
#disply all the columns
pd.set_option('display.max_columns', None)

In [6]:
invoice_All.head()

,ID,SiteUID,Date,OutletUID,outlet,ProvinceUID,province,DistrictUID,district,AreaUID,Area,AgentUID,AgentName,DistributorUID,distributor,ItemUID,item,UnitQty,UnitPrice,RetailUnitPrice,FreeQty,FreeCompanyQty,GrossValue,Year,MonthName,Month,Day,WeekDay,DateName,Hour
0,CSO008001523,56,2018-11-07 07:59:37.407,377,SETHSIRI PHARMACY,24.0,North Western Province,26.0,Puttalam,92.0,CHILAW,80,Sithum Asela,2,The West Central Pharma (Pvt) Ltd.,238,Dicloran Gel 20gm,10.0,170.60,194.5,0.0,0.0,1706.0,2018,November,11,7,4,Wednesday,7
1,CSO008001523,56,2018-11-07 07:59:37.407,377,SETHSIRI PHARMACY,24.0,North Western Province,26.0,Puttalam,92.0,CHILAW,80,Sithum Asela,2,The West Central Pharma (Pvt) Ltd.,238,Dicloran Gel 20gm,0.0,170.60,194.5,1.0,1.0,0.0,2018,November,11,7,4,Wednesday,7
2,CSO007901614,53,2018-11-07 08:04:10.913,389,UNION PHARMACY,24.0,North Western Province,26.0,Puttalam,92.0,CHILAW,79,Sumedha Nilanka,2,The West Central Pharma (Pvt) Ltd.,109,Coloncare - Capsules (3x10's),1.0,1018.40,1161.0,0.0,0.0,1018.4,2018,November,11,7,4,Wednesday,8
3,CSO007700418,71,2018-11-07 08:05:52.420,123,S S I Pharmacy,6.0,Eastern Province,7.0,Ampara,84.0,Nintavur,77,Mohamed Ramees,1,Apex Med Pharmaceuticals,104,Xtar Toothpaste 100gm,2.0,607.80,693.0,0.0,0.0,1215.6,2018,November,11,7,4,Wednesday,8
4,CSO008701116,66,2018-11-07 11:29:50.177,1185,Laksanda Pharmacy,2.0,Central Province,3.0,Kandy,211.0,Peradeniya,87,Sadish Kanna,6,J.G.P Marketing (Pvt) Ltd,234,Nicardia-XL-30mg 30's,2.0,705.25,804.0,0.0,0.0,1410.5,2018,November,11,7,4,Wednesday,11


In [6]:
invoice_All.dtypes

ID                         object
SiteUID                     int64
Date               datetime64[ns]
OutletUID                   int64
outlet                     object
ProvinceUID               float64
province                   object
DistrictUID               float64
district                   object
AreaUID                   float64
Area                       object
AgentUID                    int64
AgentName                  object
DistributorUID              int64
distributor                object
ItemUID                     int64
item                       object
UnitQty                   float64
UnitPrice                 float64
RetailUnitPrice           float64
FreeQty                   float64
FreeCompanyQty            float64
GrossValue                float64
Year                       object
MonthName                  object
Month                       int64
Day                        object
WeekDay                     int64
DateName                   object
Hour          

In [7]:
invoice_All.describe()

,SiteUID,OutletUID,ProvinceUID,DistrictUID,AreaUID,AgentUID,DistributorUID,ItemUID,UnitQty,UnitPrice,RetailUnitPrice,FreeQty,FreeCompanyQty,GrossValue,Month,WeekDay,Hour
count,1.126154e+06,1.126154e+06,1.126143e+06,1.126143e+06,1.126143e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06,1.126154e+06
mean,2.475577e+02,1.924469e+03,1.919109e+01,2.089609e+01,2.340823e+02,1.174024e+02,1.223078e+01,1.564247e+02,8.821112e+00,7.193981e+02,8.272034e+02,1.232117e+00,1.250152e+00,3.634456e+03,6.469802e+00,4.318789e+00,1.254347e+01
std,1.688518e+02,1.327379e+03,7.814580e+00,7.818096e+00,1.356535e+02,3.267446e+01,7.529730e+00,1.295727e+02,7.428661e+01,8.555185e+02,9.839469e+02,2.620872e+01,2.620671e+01,1.897601e+04,3.755982e+00,1.677176e+00,3.382479e+00
min,0.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+01,7.700000e+01,1.000000e+00,1.000000e+00,0.000000e+00,2.630000e+01,3.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,9.800000e+01,8.240000e+02,1.600000e+01,1.800000e+01,1.050000e+02,9.200000e+01,6.000000e+00,3.900000e+01,1.000000e+00,2.500000e+02,2.850000e+02,0.000000e+00,0.000000e+00,5.544000e+02,3.000000e+00,3.000000e+00,1.000000e+01
50%,2.270000e+02,1.698000e+03,2.000000e+01,2.100000e+01,2.220000e+02,1.050000e+02,1.200000e+01,1.310000e+02,2.000000e+00,3.730000e+02,4.360000e+02,0.000000e+00,0.000000e+00,1.342200e+03,7.000000e+00,4.000000e+00,1.300000e+01
75%,3.540000e+02,2.672000e+03,2.400000e+01,2.500000e+01,3.430000e+02,1.550000e+02,1.700000e+01,2.440000e+02,6.000000e+00,8.700000e+02,9.940000e+02,0.000000e+00,0.000000e+00,2.890000e+03,1.000000e+01,6.000000e+00,1.500000e+01
max,6.660000e+02,6.116000e+03,3.300000e+01,3.500000e+01,5.300000e+02,1.960000e+02,2.900000e+01,5.810000e+02,5.217300e+04,1.784800e+04,2.088200e+04,1.000000e+04,1.000000e+04,8.467678e+06,1.200000e+01,7.000000e+00,2.300000e+01
